## Train model
Danh sách class: <br>
    0. Correct <br>
    1. Chân quá hẹp <br >
    2. Chân quá rộng <br >
    3. Khoảng cách giữa 2 đầu gối quá nhỏ <br >
    4. Xuống quá xâu <br >
    5. Lỗi gập gập lưng <br >

In [1]:
import mediapipe as mp
import cv2
import pandas as pd
import pickle
import numpy as np
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 1. Khởi tạo keypoints và các phương thức 
Các keypoints cần lấy :
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",

Dataframe gồm label và các features là toạ độ keypoints gồm 4 thành phần: x, y, z, visibility
- x, y: tọa độ x, y của keypoints đã được chuẩn hóa min max scale
- z: độ sâu của điểm ảnh theo điểm giữa hông 
- visibility: biểu thị khả năng có thể nhìn thấy hoặc che khuất
 

In [2]:
#Chọn các điểm cần lấy
Important_kp = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
]

#Tạo header cho dataframe
header = ["label"]

for kp in Important_kp:
    header.extend([f"{kp}_x", f"{kp}_y", f"{kp}_z", f"{kp}_visibility"])

header

['label',
 'NOSE_x',
 'NOSE_y',
 'NOSE_z',
 'NOSE_visibility',
 'LEFT_SHOULDER_x',
 'LEFT_SHOULDER_y',
 'LEFT_SHOULDER_z',
 'LEFT_SHOULDER_visibility',
 'RIGHT_SHOULDER_x',
 'RIGHT_SHOULDER_y',
 'RIGHT_SHOULDER_z',
 'RIGHT_SHOULDER_visibility',
 'LEFT_HIP_x',
 'LEFT_HIP_y',
 'LEFT_HIP_z',
 'LEFT_HIP_visibility',
 'RIGHT_HIP_x',
 'RIGHT_HIP_y',
 'RIGHT_HIP_z',
 'RIGHT_HIP_visibility',
 'LEFT_KNEE_x',
 'LEFT_KNEE_y',
 'LEFT_KNEE_z',
 'LEFT_KNEE_visibility',
 'RIGHT_KNEE_x',
 'RIGHT_KNEE_y',
 'RIGHT_KNEE_z',
 'RIGHT_KNEE_visibility',
 'LEFT_ANKLE_x',
 'LEFT_ANKLE_y',
 'LEFT_ANKLE_z',
 'LEFT_ANKLE_visibility',
 'RIGHT_ANKLE_x',
 'RIGHT_ANKLE_y',
 'RIGHT_ANKLE_z',
 'RIGHT_ANKLE_visibility']

In [3]:
from math import sqrt

# rescale frame 1/2
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

#Tính khoảng cách giữa 2 điểm trong không gian 2D
def calculate_distance(pointX, pointY) -> float:
    '''
    Calculate a distance between 2 points
    '''

    x1, y1 = pointX
    x2, y2 = pointY

    return sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

#Tính góc giữa 3 điểm trong không gian 3D
def calculate_angle(pointA, pointB, pointC) -> float:
    """
    Tính góc giữa ba điểm trong không gian 3D (đơn vị: độ)
    a, b, c là danh sách chứa tọa độ 3D: [x, y, z]
    """

    #Độ sâu ảnh
    t = 33;
    A = np.array([pointA[0], pointA[1], pointA[2]/2])  # (xA, yA, zA)
    B = np.array([pointB[0], pointB[1], pointB[2]/2 ])  # (xB, yB, zB)
    C = np.array([pointC[0], pointC[1], pointC[2]/2])  # (xC, yC, zC)

    # Vector BA và BC
    BA = A - B
    BC = C - B

    # Tính tích vô hướng
    dot_product = np.dot(BA, BC)
    
    # Tính độ dài vector
    norm_BA = np.linalg.norm(BA)
    norm_BC = np.linalg.norm(BC)

    # Tính góc bằng công thức cos(theta) = (A.B) / (|A| * |B|)
    cos_theta = dot_product / (norm_BA * norm_BC)
    
    # Chuyển từ radian sang độ
    angle = np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

    return angle

#Đọc dữ liệu từ frame trả về points
def extract_important_keypoints(results) -> list:
    '''
    Extract important keypoints from mediapipe pose detection
    '''
    landmarks = results.pose_landmarks.landmark

    data = []
    for lm in Important_kp:
        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
        data.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
    
    return np.array(data).flatten().tolist()


In [4]:
#Lỗi 1 - 2: ratio = feet/shoulder
Min_radio_feet_shoulder = 0.59
Max_radio_feet_shoulder = 1

#Lỗi 3 : ratio = knee/feet
Min_up_radio_knee_feet = 0.57
Min_middle_ratio_knee_feet = 0.67
Min_down_ratio_knee_feet = 1.1

#Lỗi 4: knee angle < 50
Min_knee_angle = 50

#Lỗi 5: ratio = knee_angle/hip_angle
Max_ratio_knee_hip = 0.98


In [5]:
df = pd.DataFrame(columns=header)
df_test = pd.DataFrame(columns=header)
df_va = pd.DataFrame(columns=header)
df 

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility


# 2. Hàm duyệt data lỗi chân quá hẹp

In [22]:
#Xử lý từng video để tính khoảng cách chân, vai
def detection_1_error(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    images = []  # Danh sách lưu hình ảnh
    max_images = 10  # Số hình ảnh tối đa hiển thị cùng lúc

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích xuất kpkp
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            #khởi tạo biến tính khoảng cách
            shoulder_width = feet_width = None
            
            try:
                # Lấy kp từ frame
                # row = extract_important_keypoints(results)
                # X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh khoảng cách vai, hông, chân
                landmarks = results.pose_landmarks.landmark
                # Khoảng cách giữa 2 vai
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                shoulder_width = calculate_distance(left_shoulder, right_shoulder)

                # Khoảng cách giữa 2 chân (sửa từ foot_index thành ankle)
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                feet_width = calculate_distance(left_ankle, right_ankle)


                # Kiểm tra lỗi
                row = []
                if(feet_width/shoulder_width < Min_radio_feet_shoulder):
                    row.append(1)
                    for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])

                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display feet distance
                cv2.putText(image, "FEET", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(feet_width, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display shoulder distance
                cv2.putText(image, "SHOULDER", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(shoulder_width, 2)), (90, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display label
                cv2.putText(image, "LABEL", (200, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(row[0], 2)), (200, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            images.append(image)

            # Khi đủ 10 ảnh, hiển thị tất cả cùng lúc
            # if len(images) == max_images:
            #     fig, axes = plt.subplots(5, 2, figsize=(32, 18))  # Tạo grid 2x5
            #     for i, ax in enumerate(axes.flat):
            #         ax.imshow(images[i])
            #         ax.set_aspect('auto')
            #         ax.axis("off")  # Ẩn trục tọa độ
            #     plt.show()
            #     images = []  # Reset danh sách sau khi hiển thị

        Cap.release()

In [23]:
import os

video_folder = "Data/train/Chan_qua_hep"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_1_error(video_folder, video_file, "train")

In [37]:
df 

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility
0,1.0,0.535062,0.249332,-0.298349,0.999890,0.567664,0.323744,-0.071113,0.999684,0.477610,...,-0.019138,0.992065,0.558640,0.903493,0.080078,0.970077,0.513414,0.908219,0.124308,0.983346
1,1.0,0.535019,0.256015,-0.305654,0.999889,0.567926,0.326266,-0.081536,0.999688,0.478063,...,-0.040403,0.992278,0.558611,0.903518,0.091800,0.970177,0.512646,0.909850,0.102087,0.983742
2,1.0,0.535222,0.263654,-0.307573,0.999891,0.567934,0.334535,-0.082262,0.999697,0.478092,...,-0.034905,0.992214,0.558633,0.901283,0.092424,0.970170,0.512343,0.909795,0.107190,0.983548
3,1.0,0.535212,0.279500,-0.308483,0.999881,0.567779,0.343994,-0.084086,0.999694,0.478039,...,-0.032856,0.991776,0.558793,0.898668,0.109657,0.968650,0.512348,0.909864,0.111147,0.982737
4,1.0,0.535222,0.287921,-0.310271,0.999887,0.567733,0.356836,-0.086673,0.999710,0.477941,...,-0.026602,0.991716,0.558939,0.897077,0.102962,0.969260,0.512467,0.910724,0.133688,0.982723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552,1.0,0.502052,0.164798,-0.200236,0.999942,0.555950,0.271383,-0.066368,0.999930,0.462536,...,0.026470,0.987777,0.514152,0.882497,0.113064,0.981108,0.476988,0.877122,0.165850,0.985852
1553,1.0,0.502024,0.164827,-0.197816,0.999942,0.555996,0.271385,-0.066375,0.999930,0.462649,...,0.028332,0.987655,0.514392,0.884180,0.110009,0.981414,0.476964,0.877098,0.166238,0.985932
1554,1.0,0.502019,0.164854,-0.198327,0.999941,0.556103,0.271424,-0.067913,0.999929,0.462671,...,0.028563,0.987513,0.514563,0.883134,0.114635,0.981558,0.477222,0.875442,0.167355,0.985941
1555,1.0,0.502188,0.164731,-0.202035,0.999941,0.556240,0.271480,-0.069484,0.999928,0.462673,...,0.028522,0.987413,0.514693,0.881331,0.117688,0.981581,0.477643,0.874598,0.169866,0.985906


# 2. Lỗi chân rộng


In [7]:
#Xử lý từng video để tính khoảng cách chân, vai
def detection_2_error(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    images = []  # Danh sách lưu hình ảnh
    max_images = 10  # Số hình ảnh tối đa hiển thị cùng lúc

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích xuất kpkp
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            #khởi tạo biến tính khoảng cách
            shoulder_width = feet_width = None
            
            try:
                # Lấy kp từ frame
                # row = extract_important_keypoints(results)
                # X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh khoảng cách vai, hông, chân
                landmarks = results.pose_landmarks.landmark
                # Khoảng cách giữa 2 vai
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                shoulder_width = calculate_distance(left_shoulder, right_shoulder)

                # Khoảng cách giữa 2 chân
                left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
                right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]

                feet_width = calculate_distance(left_foot_index, right_foot_index)

                # Kiểm tra lỗi
                row = []
                if(feet_width/shoulder_width > Max_radio_feet_shoulder):
                    row.append(2)
                    for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])

                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display feet distance
                cv2.putText(image, "FEET", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(feet_width, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display shoulder distance
                cv2.putText(image, "SHOULDER", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(shoulder_width, 2)), (90, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display label
                cv2.putText(image, "LABEL", (200, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(row[0], 2)), (200, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            images.append(image)

            # Khi đủ 10 ảnh, hiển thị tất cả cùng lúc
            # if len(images) == max_images:
            #     fig, axes = plt.subplots(5, 2, figsize=(32, 18))  # Tạo grid 2x5
            #     for i, ax in enumerate(axes.flat):
            #         ax.imshow(images[i])
            #         ax.set_aspect('auto')
            #         ax.axis("off")  # Ẩn trục tọa độ
            #     plt.show()
            #     images = []  # Reset danh sách sau khi hiển thị

        Cap.release()

In [40]:
import os

video_folder = "Data/train/Chan_qua_rong"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_2_error(video_folder, video_file, "train")

In [41]:
df

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility
0,2.0,0.459091,0.112107,-0.313235,0.999997,0.516672,0.213783,-0.140439,0.999987,0.423691,...,0.072711,0.997105,0.526714,0.808793,0.092192,0.998317,0.406988,0.793673,0.262605,0.997580
1,2.0,0.459138,0.113000,-0.295595,0.999997,0.516637,0.215086,-0.130685,0.999987,0.423109,...,0.078769,0.997144,0.527438,0.808834,0.129791,0.998289,0.407018,0.793917,0.277647,0.997600
2,2.0,0.459202,0.113371,-0.279465,0.999997,0.516548,0.215081,-0.114115,0.999987,0.422708,...,0.078412,0.997161,0.527564,0.810169,0.122766,0.998231,0.407052,0.793974,0.275216,0.997588
3,2.0,0.459339,0.113615,-0.277441,0.999996,0.516547,0.214897,-0.112786,0.999987,0.422234,...,0.077627,0.997159,0.527763,0.810173,0.121305,0.998156,0.407101,0.794454,0.270584,0.997568
4,2.0,0.459416,0.113755,-0.275614,0.999996,0.516557,0.214649,-0.111334,0.999986,0.421924,...,0.076668,0.997196,0.527798,0.810194,0.116951,0.998125,0.407113,0.794519,0.270677,0.997583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,2.0,0.515885,0.135050,-0.229636,0.999994,0.561906,0.250951,-0.025717,0.999979,0.467342,...,-0.006169,0.997307,0.582271,0.885328,0.187713,0.994357,0.453820,0.890234,0.174316,0.997102
1493,2.0,0.515551,0.135061,-0.229236,0.999994,0.561803,0.250958,-0.027120,0.999980,0.467330,...,-0.005158,0.997308,0.582204,0.885200,0.186860,0.994346,0.453821,0.889975,0.176889,0.997099
1494,2.0,0.515401,0.135066,-0.221628,0.999994,0.561670,0.250983,-0.024165,0.999980,0.467332,...,-0.005144,0.997310,0.582151,0.885000,0.184367,0.994354,0.453825,0.889823,0.176811,0.997092
1495,2.0,0.515334,0.135069,-0.214825,0.999994,0.561577,0.251021,-0.019305,0.999981,0.467340,...,-0.005611,0.997310,0.582117,0.884941,0.183298,0.994332,0.453815,0.889763,0.175257,0.997080


# 3. Lỗi gối hẹp


In [33]:
#import model để dự đoán giai đoạn
import pickle


with open("LR_Up_Down_model.pkl", "rb") as f:
    trained_model = pickle.load(f)

In [ ]:
#Xử lý từng video để tính khoảng cách chân, vai
def detection_3_error(Video_folder = "Data/train/Dau_goi_hep" , Video_name = "20250228_092009000_iOS.mp4" , type = "train"):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích xuất kpkp
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            # Chuyển lại ảnh sang BGR để hiển thị
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #khởi tạo biến tính khoảng cách
            knee_width = feet_width = None
            
            try:
                # Lấy kp từ frame
                row = extract_important_keypoints(results)
                X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh khoảng cách đầu gối, chân
                landmarks = results.pose_landmarks.landmark

                # Lấy trạng thái hiện tại 
                predicted_class = trained_model.predict(X)[0]
                prediction_probability = trained_model.predict_proba(X)[0]

                current_stage ="middle"
                if prediction_probability[prediction_probability.argmax()] >= 0.5: # Nếu xác suất dự đoán lớn hơn 0.5 thì mới cập nhật trạng thái
                    if predicted_class == 0:
                        current_stage = "down"
                    elif current_stage == "middle" and predicted_class == 1:
                        current_stage = "up"
                else:
                    current_stage = "middle"

                # Khoảng cách giữa 2 đầu gối 
                left_knee  = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

                knee_width = calculate_distance(left_knee, right_knee)

                # Khoảng cách giữa 2 chân
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                feet_width = calculate_distance(left_ankle, right_ankle)

                ratio = knee_width/feet_width

                # Kiểm tra lỗi
                row = []
                isError = False
                if(current_stage == "down" and ratio < Min_down_ratio_knee_feet):
                    isError = True
                elif(current_stage == "middle" and ratio < Min_middle_ratio_knee_feet):
                    isError = True
                elif(current_stage == "up" and ratio < Min_up_radio_knee_feet):
                    isError = True

                if isError:
                    row.append(3)
                    for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])

                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")


                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display feet distance
                cv2.putText(image, "FEET", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(feet_width, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display knee distance
                cv2.putText(image, "KNEE", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(knee_width, 2)), (90, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display label
                cv2.putText(image, "LABEL", (200, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(row[0], 2)), (200, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display current stage
                cv2.putText(image, "STAGE", (300, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, current_stage, (300, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)


            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            # Hiển thị ảnh
            plt.clf()  # Xóa hình ảnh cũ
            plt.imshow(image)
            plt.axis("off")  # Ẩn trục tọa độ
            plt.show()

        Cap.release()

In [ ]:
import os

df = pd.DataFrame(columns=header)
df_test = pd.DataFrame(columns=header)
df_va = pd.DataFrame(columns=header)

#Path
video_folder = "Data/train/Dau_goi_hep"
# video_folder = "Data/train/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_3_error(video_folder, video_file, "train")

In [46]:
df 

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility
0,3.0,0.504455,0.394290,-0.300894,0.999950,0.541155,0.451518,-0.136166,0.999856,0.464396,...,-0.082335,0.994534,0.519857,0.845551,0.009311,0.985591,0.466354,0.852010,0.042395,0.992992
1,3.0,0.504455,0.400474,-0.298456,0.999951,0.541101,0.458860,-0.133709,0.999860,0.464405,...,-0.118449,0.994316,0.519738,0.845618,-0.008733,0.985043,0.466351,0.852306,0.010387,0.992628
2,3.0,0.504204,0.410367,-0.288947,0.999952,0.541064,0.467339,-0.131028,0.999862,0.464449,...,-0.165305,0.994037,0.519596,0.845645,-0.035462,0.984005,0.466347,0.852411,-0.026965,0.992189
3,3.0,0.504391,0.421738,-0.288882,0.999952,0.541002,0.475882,-0.132371,0.999864,0.464482,...,-0.180093,0.993585,0.519435,0.845673,-0.049492,0.982862,0.466292,0.852459,-0.037647,0.991743
4,3.0,0.504840,0.432081,-0.282700,0.999953,0.540929,0.485532,-0.131406,0.999867,0.464495,...,-0.194537,0.993259,0.519240,0.846213,-0.060424,0.982150,0.466204,0.852691,-0.050264,0.991557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,3.0,0.453069,0.323035,-0.327240,0.999455,0.491285,0.405923,-0.139664,0.999456,0.397281,...,-0.287149,0.939423,0.471389,0.805647,-0.068251,0.881847,0.418719,0.811787,-0.061664,0.905567
90,3.0,0.452476,0.309446,-0.334103,0.999494,0.491279,0.393350,-0.143433,0.999492,0.397067,...,-0.280607,0.942277,0.471798,0.805553,-0.077354,0.889864,0.418767,0.811792,-0.064813,0.911918
91,3.0,0.452111,0.296225,-0.343269,0.999538,0.491276,0.383113,-0.149258,0.999525,0.396700,...,-0.284974,0.945341,0.471917,0.806182,-0.070974,0.897011,0.418718,0.812109,-0.074172,0.918401
92,3.0,0.451585,0.283207,-0.359224,0.999577,0.491125,0.372128,-0.161785,0.999559,0.396091,...,-0.246141,0.947230,0.472036,0.806179,-0.051538,0.903827,0.418717,0.812094,-0.017729,0.923611


# 4.Lỗi xuống quá xâu

In [53]:
#Xử lý từng video để tính khoảng cách chân, đàu gốin 
def detection_4_error(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích frame như ảnh đầu vào
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            # Chuyển lại ảnh sang BGR để hiển thị
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #khởi tạo biến tính
            knee_angle = None
            hip_angle = None

            
            try:
                # Lấy kp từ frame
                row = extract_important_keypoints(results)
                X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh góc giữa 2 đầu gối
                landmarks = results.pose_landmarks.landmark

                # Góc đầu gối trái 
                left_knee  = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
                left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

                # Góc đầu gối phải
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]

                right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)

                #Láy trung bình cộng 2 góc
                knee_angle = (left_knee_angle + right_knee_angle) / 2

                row = []
                if(knee_angle < Min_knee_angle):
                    row.append(4)
                    for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
                
                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                #Display label
                cv2.putText(image, "LABEL", (10, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(row[0], 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display knee angle 
                cv2.putText(image, "knee_angle", (150, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(knee_angle, 2)), (150, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            # if(knee_angle < 55):
            #     plt.imshow(image)
            #     plt.axis("off")  # Ẩn trục tọa độ
            #     plt.show()

        Cap.release()

# Đường dẫn thư mục chứa video
# video_folder = "Data/Train/Correct"
# video_files = "20250228_150821.mp4"

# video_folder = "Data/Train/Gap_lung"
# video_files = "20250228_165326.mp4"

# video_folder = "Data/Train/Xuong_qua_xau"
# video_files = "20250228_163416.mp4"

# process_frame(video_folder, video_files)



In [54]:
import os

df = pd.DataFrame(columns=header)
df_test = pd.DataFrame(columns=header)
df_va = pd.DataFrame(columns=header)

#Path
video_folder = "Data/train/Xuong_qua_xau"
# video_folder = "Data/train/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_4_error(video_folder, video_file, "train")

Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: list index out of range
Lỗi! row có 0 phần tử nhưng df có 37 cột.
Error: li

In [55]:
df

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility
0,4.0,0.513528,0.462066,-0.108319,0.999698,0.556474,0.542042,0.016325,0.993081,0.465201,...,-0.268669,0.986376,0.557614,0.890369,-0.161493,0.940670,0.475349,0.891248,-0.136326,0.972703
1,4.0,0.513477,0.464205,-0.108197,0.999706,0.556483,0.544296,0.016203,0.993218,0.465261,...,-0.264462,0.985839,0.557700,0.890570,-0.161197,0.939767,0.475346,0.891688,-0.133545,0.971975
2,4.0,0.513459,0.466149,-0.108132,0.999716,0.556485,0.545880,0.016297,0.993409,0.465304,...,-0.261406,0.985389,0.557701,0.890843,-0.161637,0.939193,0.475341,0.892164,-0.130318,0.971255
3,4.0,0.513448,0.468059,-0.107091,0.999725,0.556496,0.547768,0.017133,0.993544,0.465349,...,-0.262290,0.985036,0.557721,0.891168,-0.161189,0.938709,0.475347,0.892785,-0.131143,0.970825
4,4.0,0.513580,0.469192,-0.106753,0.999729,0.556559,0.549989,0.017740,0.993578,0.465364,...,-0.262498,0.984662,0.557759,0.891605,-0.160930,0.938194,0.475415,0.893720,-0.131720,0.970356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,4.0,0.463978,0.546362,-0.165078,0.999990,0.518586,0.604343,-0.070372,0.999863,0.435606,...,-0.145917,0.978123,0.510143,0.872913,-0.079048,0.963040,0.437494,0.872825,-0.041738,0.974135
478,4.0,0.463954,0.542993,-0.165385,0.999990,0.518464,0.602515,-0.070386,0.999866,0.435413,...,-0.146875,0.978710,0.510139,0.872550,-0.080700,0.963578,0.437481,0.872482,-0.042048,0.974747
479,4.0,0.463898,0.540189,-0.168489,0.999990,0.518464,0.601230,-0.072282,0.999869,0.435023,...,-0.146215,0.979377,0.510136,0.872262,-0.081807,0.964360,0.437439,0.872199,-0.042197,0.975486
480,4.0,0.463660,0.538271,-0.178073,0.999991,0.518383,0.599891,-0.079325,0.999869,0.434347,...,-0.146643,0.979848,0.510132,0.871968,-0.082153,0.964722,0.437431,0.872087,-0.043085,0.976026


# 5. Lỗi gập lưng
